In [1]:
import pycuda.autoinit
from pycuda.compiler import SourceModule
import pycuda.driver as cuda
import pycuda.gpuarray as gpuarray
import numpy as np
import time
import matplotlib.pyplot as plt
import math

In [179]:
class PrefixSum:
    def __init__(self):
        self.mod = None
        self.getSourceModule()


    def getSourceModule(self):
        kernelwrapper = """
        //------------------------------------------------------------------------------
        // predefined parameters
        #define SECTION_SIZE 1024

        //------------------------------------------------------------------------------

        // Kogge-Stone scan kernel, naive-gpu
        __global__
        void KoggeStone(float *X, float *Y, const int InputSize){
            
            // copy the input array to shared memory
            __shared__ float XY[SECTION_SIZE];

            const int tx = threadIdx.x;
            const int i = blockDim.x*blockIdx.x + tx;

            if(i<InputSize){
                XY[tx] = X[i];
            }

            // iterative scan on XY
            float temp = 0.0f;
            for(unsigned int stride = 1; stride < blockDim.x; stride*=2){
                temp = 0;
                __syncthreads();
                if(tx >= stride) temp = XY[tx-stride];
                __syncthreads();
                XY[tx] += temp;
                //if(tx >= stride) XY[tx] += XY[tx-stride];
            }
            __syncthreads();
        
            if(i<InputSize) Y[i] = XY[tx];
            //Y[i] = XY[tx];

        }

    
        //------------------------------------------------------------------------------

        // Kogge-Stone scan kernel for arbitrary input length, naive-gpu
        // phase1&2 for inefficient scan
        __global__
        void KoggeStone_end(double *X, double *Y, double *end_ary, const int InputSize){
            
            // copy the input array to shared memory
            __shared__ double XY[SECTION_SIZE];

            const int tx = threadIdx.x;
            const unsigned long int i = blockDim.x*blockIdx.x + tx;

            if(i<InputSize){
                XY[tx] = X[i];
            }

            // iterative scan on XY
            float temp = 0.0f;
            for(unsigned int stride = 1; stride < blockDim.x; stride*=2){
                temp = 0;
                __syncthreads();
                if(tx >= stride) temp = XY[tx-stride];
                __syncthreads();
                XY[tx] += temp;
                //if(tx >= stride) XY[tx] += XY[tx-stride];
            }
            __syncthreads();
        
            if(i<InputSize) Y[i] = XY[tx];
            //Y[i] = XY[tx];

            // copy the last element of the block
            if(tx == SECTION_SIZE-1){
                end_ary[blockIdx.x] = XY[tx];
            }
            else if(i == InputSize-1){
                end_ary[blockIdx.x] = XY[tx];
            }

        }


        //------------------------------------------------------------------------------
        // phase 3 of inefficient scan

        __global__
        void phase3_koggestone(double *Y, double *S, const int InputSize){

            // current position in Y
            const unsigned long int y = blockDim.x*blockIdx.x + threadIdx.x;

            if(blockIdx.x > 0 && y<InputSize) Y[y] += S[blockIdx.x-1];
            
        }


        //------------------------------------------------------------------------------

        #define SECTION_SIZE_2 SECTION_SIZE*2

        //------------------------------------------------------------------------------

        // Brent-Kung scan kernel phase1

        __global__ 
        void BrentKung(float *X, float *Y, const int InputSize){

            __shared__ float XY[SECTION_SIZE_2];

            // current position in block
            const int tx = threadIdx.x;
            // current position in X
            const int i = 2*blockDim.x*blockIdx.x + tx;

            if(i<InputSize) XY[tx] = X[i];
            if(i+blockDim.x < InputSize) XY[tx+blockDim.x] = X[i+blockDim.x];

            // reduction tree
            for(unsigned int stride=1; stride <= blockDim.x; stride*=2){
                __syncthreads();
                int index = (tx+1)*2*stride-1;
                if(index<SECTION_SIZE_2) XY[index] += XY[index - stride];                 
            }

            // distribution tree 
            for(unsigned int stride=SECTION_SIZE_2/4; stride > 0; stride /=2){
                __syncthreads();
                int index = (tx+1)*2*stride-1;
                if(index+stride < SECTION_SIZE_2) XY[index+stride] += XY[index];
            }

            __syncthreads();

            if(i<InputSize) Y[i] = XY[tx];
            if(i+blockDim.x < InputSize) Y[i+blockDim.x] = XY[tx+blockDim.x];

        }
        
        //------------------------------------------------------------------------------
        
        // Brent-Kung scan kernel phase2
        
        __global__ 
        void BrentKung_phase2(float *Y, float *S, const int InputSize){

            __shared__ float XY[SECTION_SIZE_2];

            // current position in block
            const int tx = threadIdx.x;
            // current position in S
            const int k = blockDim.x*blockIdx.x + tx;
            // current position in Y
            const int i = (2*k+1)*SECTION_SIZE_2-1;

            if(i<InputSize) XY[tx] = Y[i];
            if(i+SECTION_SIZE_2 < InputSize) XY[tx+SECTION_SIZE_2] = Y[i+SECTION_SIZE_2];

            // reduction tree
            for(unsigned int stride=1; stride <= blockDim.x; stride*=2){
                __syncthreads();
                int index = (tx+1)*2*stride-1;
                if(index<SECTION_SIZE_2) XY[index] += XY[index - stride];                 
            }

            // distribution tree 
            for(unsigned int stride=SECTION_SIZE_2/4; stride > 0; stride /=2){
                __syncthreads();
                int index = (tx+1)*2*stride-1;
                if(index+stride < SECTION_SIZE_2) XY[index+stride] += XY[index];
            }

            __syncthreads();

            if(i<InputSize) S[i] = XY[tx];
            if(i+blockDim.x < InputSize) S[i+blockDim.x] = XY[tx+blockDim.x];

        }




        //------------------------------------------------------------------------------
        // Brent-Kung scan kernel phase1&2 (efficient scan)
        // output two array Y: pre-fix sum, end_ary: last elements of each blocok.

        __global__ 
        void BrentKung_end(double *X, double *Y,  double *end_ary, const int InputSize){

            __shared__ double XY[SECTION_SIZE_2];

            // current position in block
            const unsigned int tx = threadIdx.x;
            // current position in X
            const unsigned long int i = 2*blockDim.x*blockIdx.x + tx;

            // each thread copy 2 element to shared memory
            if(i<InputSize) XY[tx] = X[i];
            if(i+blockDim.x < InputSize) XY[tx+blockDim.x] = X[i+blockDim.x];

            // reduction tree
            for(unsigned int stride=1; stride <= blockDim.x; stride*=2){
                __syncthreads();
                int index = (tx+1)*2*stride-1;
                if(index<SECTION_SIZE_2) XY[index] += XY[index - stride];                 
            }

            // distribution tree 
            for(unsigned int stride=SECTION_SIZE_2/4; stride > 0; stride /=2){
                int index = (tx+1)*2*stride-1;
                __syncthreads();
                if(index+stride < SECTION_SIZE_2) XY[index+stride] += XY[index];
            }

            __syncthreads();

            // output the last element of each block
            if(((tx+1) == SECTION_SIZE) && (i+blockDim.x < InputSize)){
                end_ary[blockIdx.x] = XY[tx+blockDim.x];
            }
            else if((i+1) == InputSize){
                end_ary[blockIdx.x] = XY[tx];
            }
            else if((i+blockDim.x+1) == InputSize){
                end_ary[blockIdx.x] = XY[tx+blockDim.x];
            }
            
            // output each block
            if(i<InputSize) Y[i] = XY[tx];
            if(i+blockDim.x < InputSize) Y[i+blockDim.x] = XY[tx+blockDim.x];
            
        }

        //------------------------------------------------------------------------------
        // phase 3 of efficient scan

        __global__
        void phase3_brentkung(double *Y, double *S, const int InputSize){

            // current position in Y
            const unsigned long int y = 2*(blockDim.x*blockIdx.x) + threadIdx.x;

            if(2*blockIdx.x > 0 && y<InputSize) Y[y] += S[blockIdx.x-1];
            
            if(2*blockIdx.x > 0 && y+blockDim.x < InputSize) Y[y+blockDim.x] += S[blockIdx.x-1];

        }

        """
		
        mod = SourceModule(kernelwrapper)

        self.mod = mod
    
    @staticmethod
    def prefix_sum_python(N, length):
        """
		Naive prefix sum serial implementation
        yi = x0 + ... + xi

		params:
		- N: input array

		return:
        - Y: output array
		"""

        Y = np.zeros_like(N)

        for i in range(length):
            sum_temp = 0
            for j in range(i+1):
                sum_temp += N[j]
            Y[i] = sum_temp
        
        return Y

    @staticmethod
    def prefix_sum_python2(N, length):
        Y = np.zeros_like(N)

        Y[0] = N[0]
        for i in range(length-1):
            Y[i+1] = Y[i] + N[i+1]

        return Y

    def prefix_sum_gpu_naive(self, N, length):
        """
        Prefix_sum using Kogge-Stone scan kernel

		params:
		- N: input array

		return:
		- Y: result
		- time
		"""

        Y = np.zeros_like(N)

        # memory allocation
        N_d = gpuarray.to_gpu(N)
        Y_d = gpuarray.to_gpu(Y)

        # block and grid size
        blockdim = 1024
        BlockDim = (blockdim, 1, 1)
        GridDim = (length//blockdim+1, 1, 1)

        func = self.mod.get_function("KoggeStone")
        func(N_d, Y_d, np.int32(length), block=BlockDim, grid=GridDim)

        # cuda.memcpy_dtoh(Y, Y_d)
        Y = Y_d.get()
        
        return Y


    def prefix_sum_gpu_naive2(self, N, length):
        """
        Prefix_sum using Kogge-Stone scan kernel

		params:
		- N: input array

		return:
		- Y: result
		- time
		"""

        if length<=1024:
            n = 1
        else:
            n = math.ceil(math.log(length, 1024))
        print(n)

        Y_d_list = []
        E_d_list = [gpuarray.to_gpu(N)]

        blocksize = 1024
        BlockDim = (blocksize, 1, 1)
        gridsize_list = [length]

        func_red = self.mod.get_function("KoggeStone_end")
        func_dis = self.mod.get_function("phase3_koggestone")

        # reduction
        for i in range(n):

            # memory allocation for Y
            Y_d_list.append(gpuarray.zeros(gridsize_list[-1], dtype=np.float64))
            # gridsize for this step
            gridsize_list.append((gridsize_list[i]-1)//blocksize+1)
            print(gridsize_list[-1])
            # list of last elements for this step
            E_d_list.append(gpuarray.zeros(gridsize_list[-1], dtype=np.float64))

            func_red(E_d_list[i], Y_d_list[i], E_d_list[-1], np.int32(gridsize_list[i]), block=BlockDim, grid=(gridsize_list[-1],1,1))
            cuda.Context.synchronize()

        # distribution
        for i in range(n-1)[::-1]:
            func_dis(Y_d_list[i], Y_d_list[i+1], np.int32(gridsize_list[i]), block=BlockDim, grid=(gridsize_list[i+1],1,1))
            cuda.Context.synchronize()




        Y = Y_d_list[0].get().copy()
        E = E_d_list[1].get().copy()
        
        return Y, E


    def prefix_sum_gpu_work_efficient(self, N, length):
        """
        Prefix_sum using Brent-Kung scan kernel

		params:
		- N: input array

		return:
		- Y: result
		- time
		"""
        if length<=2048:
            n = 1
        else:
            n = math.ceil(math.log(length, 2048))

        Y_d_list = []
        E_d_list = [gpuarray.to_gpu(N)]

        blocksize = 1024
        BlockDim = (blocksize, 1, 1)
        gridsize_list = [length]

        # function of reduction kernel
        func_red = self.mod.get_function("BrentKung_end")
        # function of distribution kernel
        func_dis = self.mod.get_function("phase3_brentkung")

        # reduction
        for i in range(n):

            # memory allocation for Y
            Y_d_list.append(gpuarray.zeros(gridsize_list[-1], dtype=np.float64))
            # gridsize for this step
            gridsize_list.append((gridsize_list[i]-1)//(blocksize*2)+1)
            # list of last elements for this step
            E_d_list.append(gpuarray.zeros(gridsize_list[-1], dtype=np.float64))

            func_red(E_d_list[i], Y_d_list[i], E_d_list[-1], np.int32(gridsize_list[i]), block=BlockDim, grid=(gridsize_list[-1],1,1))
            cuda.Context.synchronize()

        # distribution
        for i in range(n-1)[::-1]:
            func_dis(Y_d_list[i], Y_d_list[i+1], np.int32(gridsize_list[i]), block=BlockDim, grid=(gridsize_list[i+1],1,1))
            cuda.Context.synchronize()

        Y = Y_d_list[0].get().copy()

        return Y







    def prefix_sum_gpu_work_efficient2(self, N, length):
        """
        Prefix_sum using Brent-Kung scan kernel

		params:
		- N: input array

		return:
		- Y: result
		- time
		"""

        Y = np.zeros_like(N)

        N_d = gpuarray.to_gpu(N)
        Y_d = gpuarray.to_gpu(Y)

        # block and grid size
        blockdim = 1024
        BlockDim = (blockdim, 1, 1)
        gridsize = (length-1)//(blockdim*2)+1
        GridDim = (gridsize, 1, 1)

        # end elements of each blocks
        end_ary = np.zeros(gridsize).astype(np.float64)
        end_ary_d = gpuarray.to_gpu(end_ary)
        # invoke the kernel function 
        func = self.mod.get_function("BrentKung_end")
        func_dis = self.mod.get_function("phase3_brentkung")

        func(N_d, Y_d, end_ary_d, np.int32(length), block=BlockDim, grid=GridDim)
        cuda.Context.synchronize()
        if length > 2048:
            Y_phase2 = np.zeros(gridsize).astype(np.float64)
            Y_phase2_d = gpuarray.to_gpu(Y_phase2)
            gridsize_phase2 = (gridsize-1)//(blockdim*2)+1
            GridDim_phase2 = (gridsize_phase2, 1, 1)
            end_ary_phase2 = np.zeros(gridsize_phase2).astype(np.float64)
            end_ary_phase2_d = gpuarray.to_gpu(end_ary_phase2)
            func(end_ary_d, Y_phase2_d, end_ary_phase2_d, np.int32(gridsize), block=BlockDim, grid=GridDim_phase2)
            cuda.Context.synchronize()
            if length > 2048*2048:
                Y_phase3 = np.zeros(gridsize_phase2).astype(np.float64)
                Y_phase3_d = gpuarray.to_gpu(Y_phase3)
                gridsize_phase3 = (gridsize_phase2-1)//(blockdim*2)+1
                GridDim_phase3 = (gridsize_phase3, 1, 1)
                end_ary_phase3 = np.zeros(gridsize_phase3).astype(np.float64)
                end_ary_phase3_d = gpuarray.to_gpu(end_ary_phase3)
                func(end_ary_phase2_d, Y_phase3_d, end_ary_phase3_d, np.int32(gridsize_phase2), block=BlockDim, grid=GridDim_phase3)
                cuda.Context.synchronize()
                # distribution
                func_dis(Y_phase2_d, Y_phase3_d, np.int32(length), block=BlockDim, grid=GridDim_phase2)      
                cuda.Context.synchronize()

            # distribution
            func_dis(Y_d, Y_phase2_d, np.int32(length), block=BlockDim, grid=GridDim)

        # cuda.memcpy_dtoh(Y, Y_d)
        Y = Y_d.get()
        # end_ary = end_ary_d.get() 
        # end_ary_phase2 = end_ary_phase2_d.get()
        
        return Y






    @staticmethod
    def test_prefix_sum_python(printing=True):
        """
		Test function prefix_sum_python
        yi = x0 + ... + xi

		params:
		- printing(boolean): print debug information

		return:
        - (bool): correct or not
		"""

        N = np.arange(1,6,step=1)
        check_ary = np.array([1,3,6,10,15])
        Y = PrefixSum.prefix_sum_python(N, N.shape[0])

        if np.allclose(Y,check_ary):
            if printing:
                print('Input array: ', N)
                print('Output array: ', Y)
                print('Check array: ', check_ary)
                print('Correct!')
                return True
            else:
                return True
        else:
            if printing:
                print('Input array: ', N)
                print('Output array: ', Y)
                print('Check array: ', check_ary)
                print('Incorrect!')
                return False
            else:
                return False           


    def test_prefix_sum_gpu_naive(self):
        # implement this, note you can change the function signature (arguments and return type)
        pass

    def test_prefix_sum_gpu_work_efficient(self):
        # implement this, note you can change the function signature (arguments and return type)
        pass





In [3]:
length = 1000

N = np.random.rand(length).astype(np.float32)


summer = PrefixSum()
Y_gpu_naive = summer.prefix_sum_gpu_naive(N, length)
Y_python = summer.prefix_sum_python(N, length)
print(np.allclose(Y_gpu_naive, Y_python))
# print(np.allclose(y_gpu_))
# print(end_ary)
# print(Y_gpu_efi)

True


In [188]:
length = 2**11 * 2**16

N = np.random.rand(length).astype(np.float64)

summer = PrefixSum()
# Y_gpu_efi= summer.prefix_sum_gpu_work_efficient2(N, length)
Y_gpu_efi2 = summer.prefix_sum_gpu_work_efficient(N, length)
# print(time.time()-start)
# print(Y_gpu_efi[-1])
# print(Y_gpu_efi)
# print(Y_gpu_efi.shape)
# Y_python = summer.prefix_sum_python2(N, length)
# print(np.allclose(Y_gpu_efi, Y_gpu_efi2))
# print(np.allclose(y_gpu_))
# print(end_ary)
# print(end_ary_2)
# print(Y_gpu_efi)
print(Y_gpu_efi2)
# print(Y_gpu_efi[-20:])
# print(Y_gpu_efi[2047:2050], Y_gpu_efi[-1])
# print(Y_python[-1])
# print(Y_gpu_efi[1000:1027])
# print(Y_python[1000:1027])
# print(Y_gpu_efi)
# print(Y_python)
print(N.sum())


[3.46840719e-01 1.33265669e+00 1.63186849e+00 ... 6.71041003e+07
 6.71041009e+07 6.71041016e+07]
67104101.5748475


In [187]:
length = 2**11 * 2**16

N = np.random.rand(length).astype(np.float64)

summer = PrefixSum()
# Y_gpu_efi= summer.prefix_sum_gpu_naive(N, length)
Y_gpu_efi2, E = summer.prefix_sum_gpu_naive2(N, length)

# print(np.allclose(Y_gpu_efi, Y_gpu_efi2))

# print(Y_gpu_efi)
print(Y_gpu_efi2[1013:1028])
print(Y_gpu_efi2[-10:])
print(E)

print(N.sum())

3
131072
128
1
[511.91428334 512.69835306 513.54656561 514.33229607 515.05812374
 515.79575304 515.97174694 515.99955783 516.08433455 516.7508551
 517.08265925 517.95047244 518.56988161 519.2416178  519.39244458]
[67113114.12358986 67113114.24657273 67113114.40564233 67113114.8555667
 67113115.35630976 67113115.81148148 67113116.64907447 67113116.70665126
 67113117.3568334  67113118.26613593]
[517.08265925 521.24806188 512.97319641 ... 531.25957416 509.18232564
 529.41327933]
67113118.08026347


In [19]:
math.log(2048*2000, 2048)

1.9968894804238262

In [36]:
for i in range(5):
    print(i)
for i in range(5)[::-1]:
    print(i)

0
1
2
3
4
4
3
2
1
0


In [29]:
length = 1000
N = np.random.rand(length).astype(np.float32)


summer = PrefixSum()
Y_gpu_efi = summer.prefix_sum_gpu_work_efficient(N, length)
Y_gpu_naive = summer.prefix_sum_gpu_naive(N, length)

Y_python = summer.prefix_sum_python2(N, length)


print(np.allclose(Y_gpu_naive, Y_python, rtol=1e-5))
t = (Y_gpu_naive == Y_python)
# print(t)
# print(t[t==False].shape)
print(np.allclose(Y_gpu_efi, Y_python, rtol=1e-5))
# t2 = (Y_gpu_efi == Y_python)

# print(t2[t2==False].shape)


# print(Y_gpu_naive)
# print(Y_gpu_efi)
# print(Y_python)

True
True


In [291]:
a = Y_gpu_naive[t==False] - Y_python[t==False]
print(a.max())


2.8667297


In [44]:

PrefixSum.test_prefix_sum_python()

Input array:  [1 2 3 4 5]
Output array:  [ 1  3  6 10 15]
Check array:  [ 1  3  6 10 15]
Correct!


True

In [33]:
(free,total)=cuda.mem_get_info()
print("Global memory occupancy:%f%% free"%(free*100/total))

for devicenum in range(cuda.Device.count()):
    device=cuda.Device(devicenum)
    attrs=device.get_attributes()

    #Beyond this point is just pretty printing
    print("\n===Attributes for device %d"%devicenum)
    for (key,value) in attrs.items():
        print("%s:%s"%(str(key),str(value)))
    # print(attrs)

Global memory occupancy:98.268502% free

===Attributes for device 0
ASYNC_ENGINE_COUNT:3
CAN_MAP_HOST_MEMORY:1
CLOCK_RATE:1590000
COMPUTE_CAPABILITY_MAJOR:7
COMPUTE_CAPABILITY_MINOR:5
COMPUTE_MODE:DEFAULT
CONCURRENT_KERNELS:1
ECC_ENABLED:1
GLOBAL_L1_CACHE_SUPPORTED:1
GLOBAL_MEMORY_BUS_WIDTH:256
GPU_OVERLAP:1
INTEGRATED:0
KERNEL_EXEC_TIMEOUT:1
L2_CACHE_SIZE:4194304
LOCAL_L1_CACHE_SUPPORTED:1
MANAGED_MEMORY:1
MAXIMUM_SURFACE1D_LAYERED_LAYERS:2048
MAXIMUM_SURFACE1D_LAYERED_WIDTH:32768
MAXIMUM_SURFACE1D_WIDTH:32768
MAXIMUM_SURFACE2D_HEIGHT:65536
MAXIMUM_SURFACE2D_LAYERED_HEIGHT:32768
MAXIMUM_SURFACE2D_LAYERED_LAYERS:2048
MAXIMUM_SURFACE2D_LAYERED_WIDTH:32768
MAXIMUM_SURFACE2D_WIDTH:131072
MAXIMUM_SURFACE3D_DEPTH:16384
MAXIMUM_SURFACE3D_HEIGHT:16384
MAXIMUM_SURFACE3D_WIDTH:16384
MAXIMUM_SURFACECUBEMAP_LAYERED_LAYERS:2046
MAXIMUM_SURFACECUBEMAP_LAYERED_WIDTH:32768
MAXIMUM_SURFACECUBEMAP_WIDTH:32768
MAXIMUM_TEXTURE1D_LAYERED_LAYERS:2048
MAXIMUM_TEXTURE1D_LAYERED_WIDTH:32768
MAXIMUM_TEXTURE1D_